<a href="https://colab.research.google.com/github/anujsahani01/VoiceCloning-coqui-TTS/blob/main/Voice_Cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install datasets

In [ ]:
! pip install TTS

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
! git clone https://github.com/coqui-ai/TTS.git

Cloning into 'TTS'...
remote: Enumerating objects: 30506, done.
remote: Counting objects: 100% (1187/1187), done.
remote: Compressing objects: 100% (413/413), done.
remote: Total 30506 (delta 848), reused 1021 (delta 760), pack-reused 29319
Receiving objects: 100% (30506/30506), 159.39 MiB | 17.08 MiB/s, done.
Resolving deltas: 100% (22115/22115), done.


In [ ]:
! pip install -r /content/TTS/requirements.txt

Ignoring numpy: markers 'python_version > "3.10"' don't match your environment
Ignoring numba: markers 'python_version < "3.9"' don't match your environment


In [ ]:
!git clone https://github.com/xiph/rnnoise.git

Cloning into 'rnnoise'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 420 (delta 194), reused 192 (delta 192), pack-reused 201
Receiving objects: 100% (420/420), 836.04 KiB | 11.45 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [ ]:
!pip install pyloudnorm

In [ ]:
!sudo apt-get install curl autoconf automake libtool pkg-config python-pip sox

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autoconf is already the newest version (2.71-2).
autoconf set to manually installed.
automake is already the newest version (1:1.16.5-1.3).
automake set to manually installed.
pkg-config is already the newest version (0.29.2-1ubuntu3).
curl is already the newest version (7.81.0-1ubuntu1.13).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libpython2-stdlib libpython2.7-minimal
  libpython2.7-stdlib libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1
  python-pkg-resources python-setuptools python2 python2-minimal python2.7
  python2.7-minimal
Suggested packages:
  libsox-fmt-all libtool-doc gcj-jdk python-setuptools-doc python2-doc
  python-tk python2.7-doc binfmt-support
Recommended packages:
  python2-dev
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libpython2-stdlib libpython2.7-minimal
  libpython2.7-s

In [ ]:
#Location of original wav files.
orig_waves="/content/drive/MyDrive/dataset/orignal"

In [ ]:
# import torch
# import torchaudio
# from speechbrain.pretrained import SpectralMaskEnhancement

# enhance_model = SpectralMaskEnhancement.from_hparams(
#     source="speechbrain/metricgan-plus-voicebank",
#     savedir="pretrained_models/metricgan-plus-voicebank",
# )

# from scipy.io import wavfile
# import os
# for dirs in os.listdir('/content/drive/MyDrive/dataset/orignal'):
#   dir = f'/content/drive/MyDrive/dataset/orignal/{dirs}'
#   noisy = enhance_model.load_audio(dir).unsqueeze(0)
#   # Add relative length tensor
#   enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))
#   # dir.replace('raw', 'orignal')
#   os.remove(dir)
#   # Saving enhanced signal on disk
#   path = f'/content/drive/MyDrive/dataset/orignal/{dirs}'
#   torchaudio.save(dir, enhanced.cpu(), 16000)

In [ ]:
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

/content/rnnoise
Updating build configuration files for rnnoise, please wait....
libtoolize: putting auxiliary files in '.'.
libtoolize: linking file './ltmain.sh'
libtoolize: putting macros in AC_CONFIG_MACRO_DIRS, 'm4'.
libtoolize: linking file 'm4/libtool.m4'
libtoolize: linking file 'm4/ltoptions.m4'
libtoolize: linking file 'm4/ltsugar.m4'
libtoolize: linking file 'm4/ltversion.m4'
libtoolize: linking file 'm4/lt~obsolete.m4'
configure.ac:19: installing './compile'
configure.ac:27: installing './config.guess'
configure.ac:27: installing './config.sub'
configure.ac:22: installing './install-sh'
configure.ac:22: installing './missing'
Makefile.am: installing './depcomp'
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether the compiler supports GNU C... yes
checking whet

In [ ]:
from pathlib import Path
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
#src = sys.argv[1]
rnn = "/content/rnnoise/examples/rnnoise_demo"


paths = glob.glob(os.path.join(orig_waves, '*.wav'))

for filepath in paths:
  target_filepath=Path(str(filepath).replace("orignal", "converted"))
  target_dir=os.path.dirname(target_filepath)
  #print(target_filepath)
  if (str(filepath) == str(target_filepath)):
    raise ValueError("Source and target path are identical: " + str(target_filepath))
  print("From: " + str(filepath))
  print("To: " + str(target_filepath))

# Stereo to Mono; upsample to 48000Hz
# added -G to fix gain, -v 0.95
  subprocess.run(["sox", "-G", "-v", "0.95", filepath, "48k.wav", "remix", "-", "rate", "48000"])
  subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
  subprocess.run([rnn, "temp.raw", "rnn.raw"]) # apply rnnoise
  subprocess.run(["sox", "-G", "-v", "0.95", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

  subprocess.run(["mkdir", "-p", str(target_dir)])
  subprocess.run(["sox", "rnn.wav", str(target_filepath), "remix", "-", "highpass", "100", "lowpass", "7000", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
  data, rate = sf.read(target_filepath)

# peak normalize audio to -1 dB
  peak_normalized_audio = pyln.normalize.peak(data, -1.0)

# measure the loudness first
  meter = pyln.Meter(rate) # create BS.1770 meter
  loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
  loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
  sf.write(target_filepath, data=loudness_normalized_audio, samplerate=22050)
  print("")


From: /content/drive/MyDrive/dataset/orignal/data-07.wav
To: /content/drive/MyDrive/dataset/converted/data-07.wav

From: /content/drive/MyDrive/dataset/orignal/data-03.wav
To: /content/drive/MyDrive/dataset/converted/data-03.wav

From: /content/drive/MyDrive/dataset/orignal/data-06.wav
To: /content/drive/MyDrive/dataset/converted/data-06.wav

From: /content/drive/MyDrive/dataset/orignal/data-05.wav
To: /content/drive/MyDrive/dataset/converted/data-05.wav

From: /content/drive/MyDrive/dataset/orignal/data-02.wav
To: /content/drive/MyDrive/dataset/converted/data-02.wav

From: /content/drive/MyDrive/dataset/orignal/data-01.wav
To: /content/drive/MyDrive/dataset/converted/data-01.wav

From: /content/drive/MyDrive/dataset/orignal/data-04.wav
To: /content/drive/MyDrive/dataset/converted/data-04.wav

From: /content/drive/MyDrive/dataset/orignal/data-08.wav
To: /content/drive/MyDrive/dataset/converted/data-08.wav

From: /content/drive/MyDrive/dataset/orignal/data-11.wav
To: /content/drive/MyDr

In [ ]:
import pandas as pd
import numpy as np
from datasets import DatasetDict, Dataset


df = pd.DataFrame()

lines = []
with open('/content/drive/MyDrive/dataset/dataset.txt', 'r', encoding = 'utf-8') as f:
  text  = ''.join(f)
  lines = text.split('\n')

In [ ]:
df['data'] = lines[:-1]

In [ ]:
df.head(3)

,data
0,data-07|had to.
1,"data-03|As I have seen and lived more life, I'..."
2,data-06|girls that take charge of their own li...


In [ ]:
datasets = Dataset.from_pandas(df)
datasets

Dataset({
    features: ['data'],
    num_rows: 212
})

In [ ]:
final = []
def process(data):
  # if len(data['data']) == 2:
  audio_path = data['data'].split('|')[0]
  dialogue = data['data'].split('|')[1]
  speaker = 'priyanka'

  final = f"{audio_path}|{speaker}|{dialogue}"
  return {
      'final_data' : final
  }

In [ ]:
final_dataset = datasets.map(process, batched = False, remove_columns = ['data'])
final_dataset

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

Dataset({
    features: ['final_data'],
    num_rows: 212
})

In [ ]:
df_final = pd.DataFrame()
df_final['inp'] = final_dataset['final_data']
df_final.to_csv('/content/drive/MyDrive/dataset/metadata.csv', index = False)
df_final.to_csv('metadata.csv', index = False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/metadata.csv')
df.head(5)

,inp
0,data-07|priyanka|had to.
1,data-03|priyanka|As I have seen and lived more...
2,data-06|priyanka|girls that take charge of the...
3,"data-05|priyanka|feeling rejuvenated, feeling ..."
4,"data-02|priyanka|one time, you know, five shif..."


In [ ]:
import os
%cd /content/TTS/TTS/tts/configs
# BaseDatasetConfig: defines name, formatter and path of the dataset.
from shared_configs import BaseDatasetConfig
%cd /content/

output_path = "/content/drive/MyDrive/dataset/cloned"
if not os.path.exists(output_path):
    os.makedirs(output_path)

/content/TTS/TTS/tts/configs
/content


In [ ]:
dataset_name = 'orignal/'

In [ ]:
root_path = '/content/drive/MyDrive/dataset/converted'
meta_file = '/content/drive/MyDrive/dataset/metadata.csv'

In [ ]:
code = '''
import os
import re
import xml.etree.ElementTree as ET
from glob import glob
from pathlib import Path
from typing import List

import pandas as pd
from tqdm import tqdm

########################
# DATASETS
########################


def coqui(root_path, meta_file, ignored_speakers=None):
    """Interal dataset formatter."""
    filepath = os.path.join(root_path, meta_file)
    # ensure there are 4 columns for every line
    with open(filepath, "r", encoding="utf8") as f:
        lines = f.readlines()
    num_cols = len(lines[0].split("|"))  # take the first row as reference
    for idx, line in enumerate(lines[1:]):
        if len(line.split("|")) != num_cols:
            print(f" > Missing column in line {idx + 1} -> {line.strip()}")
    # load metadata
    metadata = pd.read_csv(os.path.join(root_path, meta_file), sep="|")
    assert all(x in metadata.columns for x in ["audio_file", "text"])
    speaker_name = None if "speaker_name" in metadata.columns else "coqui"
    emotion_name = None if "emotion_name" in metadata.columns else "neutral"
    items = []
    not_found_counter = 0
    for row in metadata.itertuples():
        if speaker_name is None and ignored_speakers is not None and row.speaker_name in ignored_speakers:
            continue
        audio_path = os.path.join(root_path, row.audio_file)
        if not os.path.exists(audio_path):
            not_found_counter += 1
            continue
        items.append(
            {
                "text": row.text,
                "audio_file": audio_path,
                "speaker_name": speaker_name if speaker_name is not None else row.speaker_name,
                "emotion_name": emotion_name if emotion_name is not None else row.emotion_name,
                "root_path": root_path,
            }
        )
    if not_found_counter > 0:
        print(f" | > [!] {not_found_counter} files not found")
    return items


def tweb(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalize TWEB dataset.
    https://www.kaggle.com/bryanpark/the-world-english-bible-speech-dataset
    """
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "tweb"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("\t")
            wav_file = os.path.join(root_path, cols[0] + ".wav")
            text = cols[1]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def mozilla(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes Mozilla meta data files to TTS format"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "mozilla"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = cols[1].strip()
            text = cols[0].strip()
            wav_file = os.path.join(root_path, "wavs", wav_file)
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def mozilla_de(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes Mozilla meta data files to TTS format"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "mozilla"
    with open(txt_file, "r", encoding="ISO 8859-1") as ttf:
        for line in ttf:
            cols = line.strip().split("|")
            wav_file = cols[0].strip()
            text = cols[1].strip()
            folder_name = f"BATCH_{wav_file.split('_')[0]}_FINAL"
            wav_file = os.path.join(root_path, folder_name, wav_file)
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def mailabs(root_path, meta_files=None, ignored_speakers=None):
    """Normalizes M-AI-Labs meta data files to TTS format

    Args:
        root_path (str): root folder of the MAILAB language folder.
        meta_files (str):  list of meta files to be used in the training. If None, finds all the csv files
            recursively. Defaults to None
    """
    speaker_regex = re.compile(f"by_book{os.sep}(male|female){os.sep}(?P<speaker_name>[^{os.sep}]+){os.sep}")
    if not meta_files:
        csv_files = glob(root_path + f"{os.sep}**{os.sep}metadata.csv", recursive=True)
    else:
        csv_files = meta_files

    # meta_files = [f.strip() for f in meta_files.split(",")]
    items = []
    for csv_file in csv_files:
        if os.path.isfile(csv_file):
            txt_file = csv_file
        else:
            txt_file = os.path.join(root_path, csv_file)

        folder = os.path.dirname(txt_file)
        # determine speaker based on folder structure...
        speaker_name_match = speaker_regex.search(txt_file)
        if speaker_name_match is None:
            continue
        speaker_name = speaker_name_match.group("speaker_name")
        # ignore speakers
        if isinstance(ignored_speakers, list):
            if speaker_name in ignored_speakers:
                continue
        print(" | > {}".format(csv_file))
        with open(txt_file, "r", encoding="utf-8") as ttf:
            for line in ttf:
                cols = line.split("|")
                if not meta_files:
                    wav_file = os.path.join(folder, "wavs", cols[0] + ".wav")
                else:
                    wav_file = os.path.join(root_path, folder.replace("metadata.csv", ""), "wavs", cols[0] + ".wav")
                if os.path.isfile(wav_file):
                    text = cols[1].strip()
                    items.append(
                        {"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path}
                    )
                else:
                    # M-AI-Labs have some missing samples, so just print the warning
                    print("> File %s does not exist!" % (wav_file))
    return items


def ljspeech(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wavs", cols[0] + ".wav")
            text = cols[2]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


###############################################################################
###############################################################################

# custom formatter
def custom_formatter(root_path, meta_file, **kwargs):
    df  = pd.read_csv(meta_file)
    items = []
    speaker_name = "custom_formatter"
    for line in df['inp']:
        cols = line.split("|")
        wav_file = os.path.join(root_path, cols[0].replace(" ", "") + ".wav")
        print(wav_file)
        text = cols[2].replace(/[!@#$%^&*]/g, "")
        items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items

###############################################################################
###############################################################################


def ljspeech_test(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file for TTS testing
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    with open(txt_file, "r", encoding="utf-8") as ttf:
        speaker_id = 0
        for idx, line in enumerate(ttf):
            # 2 samples per speaker to avoid eval split issues
            if idx % 2 == 0:
                speaker_id += 1
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wavs", cols[0] + ".wav")
            text = cols[2]
            items.append(
                {"text": text, "audio_file": wav_file, "speaker_name": f"ljspeech-{speaker_id}", "root_path": root_path}
            )
    return items


def thorsten(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the thorsten meta data file to TTS format
    https://github.com/thorstenMueller/deep-learning-german-tts/"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "thorsten"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wavs", cols[0] + ".wav")
            text = cols[1]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def sam_accenture(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the sam-accenture meta data file to TTS format
    https://github.com/Sam-Accenture-Non-Binary-Voice/non-binary-voice-files"""
    xml_file = os.path.join(root_path, "voice_over_recordings", meta_file)
    xml_root = ET.parse(xml_file).getroot()
    items = []
    speaker_name = "sam_accenture"
    for item in xml_root.findall("./fileid"):
        text = item.text
        wav_file = os.path.join(root_path, "vo_voice_quality_transformation", item.get("id") + ".wav")
        if not os.path.exists(wav_file):
            print(f" [!] {wav_file} in metafile does not exist. Skipping...")
            continue
        items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def ruslan(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the RUSLAN meta data file to TTS format
    https://ruslan-corpus.github.io/"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "ruslan"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "RUSLAN", cols[0] + ".wav")
            text = cols[1]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def css10(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the CSS10 dataset file to TTS format"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "css10"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, cols[0])
            text = cols[1]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name})
    return items


def nancy(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the Nancy meta data file to TTS format"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "nancy"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            utt_id = line.split()[1]
            text = line[line.find('"') + 1 : line.rfind('"') - 1]
            wav_file = os.path.join(root_path, "wavn", utt_id + ".wav")
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name})
    return items


def common_voice(root_path, meta_file, ignored_speakers=None):
    """Normalize the common voice meta data file to TTS format."""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            if line.startswith("client_id"):
                continue
            cols = line.split("\t")
            text = cols[2]
            speaker_name = cols[0]
            # ignore speakers
            if isinstance(ignored_speakers, list):
                if speaker_name in ignored_speakers:
                    continue
            wav_file = os.path.join(root_path, "clips", cols[1].replace(".mp3", ".wav"))
            items.append(
                {"text": text, "audio_file": wav_file, "speaker_name": "MCV_" + speaker_name, "root_path": root_path}
            )
    return items


def libri_tts(root_path, meta_files=None, ignored_speakers=None):
    """https://ai.google/tools/datasets/libri-tts/"""
    items = []
    if not meta_files:
        meta_files = glob(f"{root_path}/**/*trans.tsv", recursive=True)
    else:
        if isinstance(meta_files, str):
            meta_files = [os.path.join(root_path, meta_files)]

    for meta_file in meta_files:
        _meta_file = os.path.basename(meta_file).split(".")[0]
        with open(meta_file, "r", encoding="utf-8") as ttf:
            for line in ttf:
                cols = line.split("\t")
                file_name = cols[0]
                speaker_name, chapter_id, *_ = cols[0].split("_")
                _root_path = os.path.join(root_path, f"{speaker_name}/{chapter_id}")
                wav_file = os.path.join(_root_path, file_name + ".wav")
                text = cols[2]
                # ignore speakers
                if isinstance(ignored_speakers, list):
                    if speaker_name in ignored_speakers:
                        continue
                items.append(
                    {
                        "text": text,
                        "audio_file": wav_file,
                        "speaker_name": f"LTTS_{speaker_name}",
                        "root_path": root_path,
                    }
                )
    for item in items:
        assert os.path.exists(item["audio_file"]), f" [!] wav files don't exist - {item['audio_file']}"
    return items


def custom_turkish(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "turkish-female"
    skipped_files = []
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wavs", cols[0].strip() + ".wav")
            if not os.path.exists(wav_file):
                skipped_files.append(wav_file)
                continue
            text = cols[1].strip()
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    print(f" [!] {len(skipped_files)} files skipped. They don't exist...")
    return items


# ToDo: add the dataset link when the dataset is released publicly
def brspeech(root_path, meta_file, ignored_speakers=None):
    """BRSpeech 3.0 beta"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            if line.startswith("wav_filename"):
                continue
            cols = line.split("|")
            wav_file = os.path.join(root_path, cols[0])
            text = cols[2]
            speaker_id = cols[3]
            # ignore speakers
            if isinstance(ignored_speakers, list):
                if speaker_id in ignored_speakers:
                    continue
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_id, "root_path": root_path})
    return items


def vctk(root_path, meta_files=None, wavs_path="wav48_silence_trimmed", mic="mic1", ignored_speakers=None):
    """VCTK dataset v0.92.

    URL:
        https://datashare.ed.ac.uk/bitstream/handle/10283/3443/VCTK-Corpus-0.92.zip

    This dataset has 2 recordings per speaker that are annotated with ```mic1``` and ```mic2```.
    It is believed that (😄 ) ```mic1``` files are the same as the previous version of the dataset.

    mic1:
        Audio recorded using an omni-directional microphone (DPA 4035).
        Contains very low frequency noises.
        This is the same audio released in previous versions of VCTK:
        https://doi.org/10.7488/ds/1994

    mic2:
        Audio recorded using a small diaphragm condenser microphone with
        very wide bandwidth (Sennheiser MKH 800).
        Two speakers, p280 and p315 had technical issues of the audio
        recordings using MKH 800.
    """
    file_ext = "flac"
    items = []
    meta_files = glob(f"{os.path.join(root_path,'txt')}/**/*.txt", recursive=True)
    for meta_file in meta_files:
        _, speaker_id, txt_file = os.path.relpath(meta_file, root_path).split(os.sep)
        file_id = txt_file.split(".")[0]
        # ignore speakers
        if isinstance(ignored_speakers, list):
            if speaker_id in ignored_speakers:
                continue
        with open(meta_file, "r", encoding="utf-8") as file_text:
            text = file_text.readlines()[0]
        # p280 has no mic2 recordings
        if speaker_id == "p280":
            wav_file = os.path.join(root_path, wavs_path, speaker_id, file_id + f"_mic1.{file_ext}")
        else:
            wav_file = os.path.join(root_path, wavs_path, speaker_id, file_id + f"_{mic}.{file_ext}")
        if os.path.exists(wav_file):
            items.append(
                {"text": text, "audio_file": wav_file, "speaker_name": "VCTK_" + speaker_id, "root_path": root_path}
            )
        else:
            print(f" [!] wav files don't exist - {wav_file}")
    return items


def vctk_old(root_path, meta_files=None, wavs_path="wav48", ignored_speakers=None):
    """homepages.inf.ed.ac.uk/jyamagis/release/VCTK-Corpus.tar.gz"""
    items = []
    meta_files = glob(f"{os.path.join(root_path,'txt')}/**/*.txt", recursive=True)
    for meta_file in meta_files:
        _, speaker_id, txt_file = os.path.relpath(meta_file, root_path).split(os.sep)
        file_id = txt_file.split(".")[0]
        # ignore speakers
        if isinstance(ignored_speakers, list):
            if speaker_id in ignored_speakers:
                continue
        with open(meta_file, "r", encoding="utf-8") as file_text:
            text = file_text.readlines()[0]
        wav_file = os.path.join(root_path, wavs_path, speaker_id, file_id + ".wav")
        items.append(
            {"text": text, "audio_file": wav_file, "speaker_name": "VCTK_old_" + speaker_id, "root_path": root_path}
        )
    return items


def synpaflex(root_path, metafiles=None, **kwargs):  # pylint: disable=unused-argument
    items = []
    speaker_name = "synpaflex"
    root_path = os.path.join(root_path, "")
    wav_files = glob(f"{root_path}**/*.wav", recursive=True)
    for wav_file in wav_files:
        if os.sep + "wav" + os.sep in wav_file:
            txt_file = wav_file.replace("wav", "txt")
        else:
            txt_file = os.path.join(
                os.path.dirname(wav_file), "txt", os.path.basename(wav_file).replace(".wav", ".txt")
            )
        if os.path.exists(txt_file) and os.path.exists(wav_file):
            with open(txt_file, "r", encoding="utf-8") as file_text:
                text = file_text.readlines()[0]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def open_bible(root_path, meta_files="train", ignore_digits_sentences=True, ignored_speakers=None):
    """ToDo: Refer the paper when available"""
    items = []
    split_dir = meta_files
    meta_files = glob(f"{os.path.join(root_path, split_dir)}/**/*.txt", recursive=True)
    for meta_file in meta_files:
        _, speaker_id, txt_file = os.path.relpath(meta_file, root_path).split(os.sep)
        file_id = txt_file.split(".")[0]
        # ignore speakers
        if isinstance(ignored_speakers, list):
            if speaker_id in ignored_speakers:
                continue
        with open(meta_file, "r", encoding="utf-8") as file_text:
            text = file_text.readline().replace("\n", "")
        # ignore sentences that contains digits
        if ignore_digits_sentences and any(map(str.isdigit, text)):
            continue
        wav_file = os.path.join(root_path, split_dir, speaker_id, file_id + ".flac")
        items.append({"text": text, "audio_file": wav_file, "speaker_name": "OB_" + speaker_id, "root_path": root_path})
    return items


def mls(root_path, meta_files=None, ignored_speakers=None):
    """http://www.openslr.org/94/"""
    items = []
    with open(os.path.join(root_path, meta_files), "r", encoding="utf-8") as meta:
        for line in meta:
            file, text = line.split("\t")
            text = text[:-1]
            speaker, book, *_ = file.split("_")
            wav_file = os.path.join(root_path, os.path.dirname(meta_files), "audio", speaker, book, file + ".wav")
            # ignore speakers
            if isinstance(ignored_speakers, list):
                if speaker in ignored_speakers:
                    continue
            items.append(
                {"text": text, "audio_file": wav_file, "speaker_name": "MLS_" + speaker, "root_path": root_path}
            )
    return items


# ======================================== VOX CELEB ===========================================
def voxceleb2(root_path, meta_file=None, **kwargs):  # pylint: disable=unused-argument
    """
    :param meta_file   Used only for consistency with load_tts_samples api
    """
    return _voxcel_x(root_path, meta_file, voxcel_idx="2")


def voxceleb1(root_path, meta_file=None, **kwargs):  # pylint: disable=unused-argument
    """
    :param meta_file   Used only for consistency with load_tts_samples api
    """
    return _voxcel_x(root_path, meta_file, voxcel_idx="1")


def _voxcel_x(root_path, meta_file, voxcel_idx):
    assert voxcel_idx in ["1", "2"]
    expected_count = 148_000 if voxcel_idx == "1" else 1_000_000
    voxceleb_path = Path(root_path)
    cache_to = voxceleb_path / f"metafile_voxceleb{voxcel_idx}.csv"
    cache_to.parent.mkdir(exist_ok=True)

    # if not exists meta file, crawl recursively for 'wav' files
    if meta_file is not None:
        with open(str(meta_file), "r", encoding="utf-8") as f:
            return [x.strip().split("|") for x in f.readlines()]

    elif not cache_to.exists():
        cnt = 0
        meta_data = []
        wav_files = voxceleb_path.rglob("**/*.wav")
        for path in tqdm(
            wav_files,
            desc=f"Building VoxCeleb {voxcel_idx} Meta file ... this needs to be done only once.",
            total=expected_count,
        ):
            speaker_id = str(Path(path).parent.parent.stem)
            assert speaker_id.startswith("id")
            text = None  # VoxCel does not provide transciptions, and they are not needed for training the SE
            meta_data.append(f"{text}|{path}|voxcel{voxcel_idx}_{speaker_id}\n")
            cnt += 1
        with open(str(cache_to), "w", encoding="utf-8") as f:
            f.write("".join(meta_data))
        if cnt < expected_count:
            raise ValueError(f"Found too few instances for Voxceleb. Should be around {expected_count}, is: {cnt}")

    with open(str(cache_to), "r", encoding="utf-8") as f:
        return [x.strip().split("|") for x in f.readlines()]


def emotion(root_path, meta_file, ignored_speakers=None):
    """Generic emotion dataset"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            if line.startswith("file_path"):
                continue
            cols = line.split(",")
            wav_file = os.path.join(root_path, cols[0])
            speaker_id = cols[1]
            emotion_id = cols[2].replace("\n", "")
            # ignore speakers
            if isinstance(ignored_speakers, list):
                if speaker_id in ignored_speakers:
                    continue
            items.append(
                {"audio_file": wav_file, "speaker_name": speaker_id, "emotion_name": emotion_id, "root_path": root_path}
            )
    return items


def baker(root_path: str, meta_file: str, **kwargs) -> List[List[str]]:  # pylint: disable=unused-argument
    """Normalizes the Baker meta data file to TTS format

    Args:
        root_path (str): path to the baker dataset
        meta_file (str): name of the meta dataset containing names of wav to select and the transcript of the sentence
    Returns:
        List[List[str]]: List of (text, wav_path, speaker_name) associated with each sentences
    """
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "baker"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            wav_name, text = line.rstrip("\n").split("|")
            wav_path = os.path.join(root_path, "clips_22", wav_name)
            items.append({"text": text, "audio_file": wav_path, "speaker_name": speaker_name, "root_path": root_path})
    return items


def kokoro(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Japanese single-speaker dataset from https://github.com/kaiidams/Kokoro-Speech-Dataset"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "kokoro"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wavs", cols[0] + ".wav")
            text = cols[2]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def kss(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Korean single-speaker dataset from https://www.kaggle.com/datasets/bryanpark/korean-single-speaker-speech-dataset"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "kss"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, cols[0])
            text = cols[2]  # cols[1] => 6월, cols[2] => 유월
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items
'''


with open("/content/TTS/TTS/tts/datasets/formatters.py","w", encoding = 'utf-8') as f:
  f.write(code)
f.close()

In [ ]:
dataset_config = BaseDatasetConfig(
    formatter="custom_formatter", meta_file_train= meta_file, path=root_path,)

In [ ]:
dataset_config

BaseDatasetConfig(formatter='custom_formatter', dataset_name='', path='/content/drive/MyDrive/dataset/converted', meta_file_train='/content/drive/MyDrive/dataset/metadata.csv', ignored_speakers=None, language='', phonemizer='', meta_file_val='', meta_file_attn_mask='')

In [ ]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=16,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=200,
    text_cleaner="phoneme_cleaners",
    phoneme_language="en-us",
    print_step=1000,
    lr=0.0005,
    # weight_decay = 0.01,
    use_phonemes=False,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=7000,
)

In [ ]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [ ]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

In [ ]:
%cd /content/TTS/TTS/tts/datasets
from formatters import custom_formatter
%cd /content/

/content/TTS/TTS/tts/datasets
/content


In [ ]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    formatter = custom_formatter,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size= 0.05,
)

/content/drive/MyDrive/dataset/converted/data-07.wav
/content/drive/MyDrive/dataset/converted/data-03.wav
/content/drive/MyDrive/dataset/converted/data-06.wav
/content/drive/MyDrive/dataset/converted/data-05.wav
/content/drive/MyDrive/dataset/converted/data-02.wav
/content/drive/MyDrive/dataset/converted/data-01.wav
/content/drive/MyDrive/dataset/converted/data-04.wav
/content/drive/MyDrive/dataset/converted/data-08.wav
/content/drive/MyDrive/dataset/converted/data-11.wav
/content/drive/MyDrive/dataset/converted/data-09.wav
/content/drive/MyDrive/dataset/converted/data-21.wav
/content/drive/MyDrive/dataset/converted/data-13.wav
/content/drive/MyDrive/dataset/converted/data-14.wav
/content/drive/MyDrive/dataset/converted/data-20.wav
/content/drive/MyDrive/dataset/converted/data-10.wav
/content/drive/MyDrive/dataset/converted/data-16.wav
/content/drive/MyDrive/dataset/converted/data-19.wav
/content/drive/MyDrive/dataset/converted/data-18.wav
/content/drive/MyDrive/dataset/converted/data-

In [ ]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [ ]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/dataset/cloned/run-July-22-2023_10+56AM-0000000

 > Model has 28597969 parameters


In [ ]:
trainer.fit()


 > EPOCH: 0/150
 --> /content/drive/MyDrive/dataset/cloned/run-July-22-2023_10+56AM-0000000
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2023-07-22 10:57:01) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 202
 | > Preprocessing samples
 | > Max text length: 222
 | > Min text length: 2
 | > Avg text length: 144.15841584158414
 | 
 | > Max audio length: 176201.0
 | > Min audio length: 176201.0
 | > Avg audio length: 176201.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
We make all our packaging from one hundred% recycled trash, from ocean bound plastic to landfills.
 [!] Character '%' not found in the vocabulary. Discarding it.



   --> TIME: 2023-07-22 10:57:16 -- STEP: 0/101 -- GLOBAL_STEP: 0
     | > current_lr: 1.25e-07 
     | > step_time: 11.984  (11.984008073806763)
     | > loader_time: 2.7834  (2.7834067344665527)

 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.

 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 10
 | > Preprocessing samples
 | > Max text length: 181
 | > Min text length: 98
 | > Avg text length: 148.5
 | 
 | > Max audio length: 176201.0
 | > Min audio length: 176201.0
 | > Avg audio length: 176201.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00233614444732666 (+0)
     | > avg_loss: 2.9717230796813965 (+0)
     | > avg_log_mle: 0.9682937264442444 (+0)
     | > avg_loss_dur: 2.003429353237152 (+0)



...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0018841028213500977 (+0.0)
     | > avg_loss: 0.29880087077617645 (+0.0)
     | > avg_log_mle: -0.2553394138813019 (+0.0)
     | > avg_loss_dur: 0.5541402846574783 (+0.0)

 > BEST MODEL : /content/drive/MyDrive/dataset/cloned/run-July-22-2023_10+56AM-0000000/best_model_15049.pth

 > EPOCH: 149/150
 --> /content/drive/MyDrive/dataset/cloned/run-July-22-2023_10+56AM-0000000

 > TRAINING (2023-07-22 13:33:47) 


We make all our packaging from one hundred% recycled trash, from ocean bound plastic to landfills.
 [!] Character '%' not found in the vocabulary. Discarding it.



 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0015705227851867676 (-0.0003135800361633301)
     | > avg_loss: 0.2887064963579178 (-0.010094374418258667)
     | > avg_log_mle: -0.2587181031703949 (-0.0033786892890930176)
     | > avg_loss_dur: 0.5474245995283127 (-0.006715685129165649)



 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0015705227851867676 (+0.0)
     | > avg_loss: 0.2887064963579178 (+0.0)
     | > avg_log_mle: -0.2587181031703949 (+0.0)
     | > avg_loss_dur: 0.5474245995283127 (+0.0)

 > BEST MODEL : /content/drive/MyDrive/dataset/cloned/run-July-22-2023_10+56AM-0000000/best_model_15150.pth


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# !pip install tensorboard
# !tensorboard --logdir=output_patha

In [ ]:
import glob, os
output_path = "/content/drive/MyDrive/dataset/cloned"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
 !tts --text "what is she doing in her room?" \
      --model_path  /content/drive/MyDrive/dataset/cloned/run-July-22-2023_10+56AM-0000000/best_model_15150.pth \
      --config_path /content/drive/MyDrive/dataset/cloned/run-July-22-2023_10+56AM-0000000/config.json \
      --out_path out.wav

 > Using model: glow_tts
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: what is she doing in her room?
 > Text splitted to sentences.
['what is she doing in her room?']
 > Processing time: 0.8253462314605713
 > Real-time factor: 0.5004092719892652
 > Saving output to out.wav


In [ ]:
! pip install pydub

In [ ]:
from pydub import AudioSegment as am
sound = am.from_file('out.wav', format='wav', frame_rate=16000)
# sound = sound.set_frame_rate(8000)
# sound.export('out.wav', format='wav')

In [ ]:
sound